# STM Transit Delay Data Preparation

## Overview

This notebook merges data from different sources and prepares it for data analysis and preprocessing.

## Data description

### Real-time Trip Updates

`current_time`: Timestamp when the data was fetched from the GTFS, in microseconds.<br>
`trip_id`: Unique identifier for the transit trip.<br>
`route_id`: Unique identifier for a bus or metro line.<br>
`start_date`: Start date of the transit trip.<br>
`stop_id`: Unique identifier of a stop.<br>
`arrival_time`, `departure_time`: Realtime arrival and departure time, in microseconds<br>
`schedule_relationship`: State of the trip, 0 meaning "scheduled", 1 meaning "skipped" and 2 meaning "no data".

### Scheduled STM Trips

`trip_id`: Unique identifier for the transit trip.<br>
`arrival_time`, `departure_time`: Scheduled arrival and departure time.<br>
`stop_id`: Unique identifier of a stop.<br>
`stop_sequence`: Sequence of a stop, for ordering.

### STM Stops

`stop_id`: Unique identifier of a stop.<br>
`stop_code`: Bus stop or metro station number.<br>
`stop_name`: Bus stop or metro station name<br>
`stop_lat`, `stop_lon`: Stop coordinates.<br>
`stop_url`: Stop web page.<br>
`location_type`: Stop type.<br>
`parent_station`: Parent station (metro station with multiple exits).<br>
`wheelchair_boarding`: Indicates if the stop is accessible for people in wheelchair, 1 being true and 2 being false.

### Real-time Vehicle Positions

`current_time`: Timestamp when the data was fetched from the GTFS, in microseconds.<br>
`vehicle_id`: Unique identifier for a vehicle.<br>
`trip_id`: Unique identifier for the transit trip.<br>
`route_id`: Unique identifier for a bus or metro line.<br>
`start_date`: Start date of a transit trip.<br>
`start_time`: Start time of a transit trip.<br>
`latitude`, `longitude`: Vehicle current position.<br>
`bearing`: Direction that the vehicle is facing, from 0 to 360 degrees.<br>
`speed`: Momentary speed measured by the vehicle, in meters per second.<br>
`stop_sequence`: Sequence of a stop, for ordering.<br>
`status`: Vehicle stop status in relation with a stop that it's currently approaching or is at, 1 being "stopped at" and 2 being "in transit to".<br>
`timestamp`: Timestamp when STM updated the data, in milliseconds.<br>
`occupancy_status`: Degree of passenger occupancy, ranging from 1 (empty) to 7 (not accepting passengers).

### Service Alerts

`start_time`: Start time of the alert, in microseconds.<br>
`end_time`: End time of the alert, in microseconds.<br>
`route_id`: Unique identifier for a bus or metro line.<br>
`stop_id`: Unique identifier of a stop.

### Weather Archive and Forecast

`time`: Date and hour or the weather.<br>
`temperature`: Air temperature at 2 meters above ground, in Celsius.<br>
`precipitation`: Total precipitation (rain, showers, snow) sum of the preceding hour, in millimeters.<br>
`windspeed`: Wind speed at 10 meters above ground, in kilometers per hour.<br>
`weathercode`: World Meteorological Organization (WMO) code, see [Open-Meteo API documentation](https://open-meteo.com/en/docs#weather_variable_documentation) for the list.

### Traffic Incidents

`category`: Category of the incident.<br>
`start_time`: Start time of the incident, in ISO8601 format.<br>
`end_time`: End time of the incident, in ISO8601 format.<br>
`length`: Length of the incident in meters.<br>
`delay`: Delay in seconds caused by the incident (except road closures).<br>
`magnitude_of_delay`: Severity of the delay, ranging from 0 to 4 (minor to major).<br>
`last_report_time`: Date when the last time the incident was reported,in ISO8601 format.<br>
`latitude`, `longitude`: Coordinates of the incident.

## Imports

In [1]:
from datetime import datetime, timedelta, timezone
from haversine import haversine, Unit
import numpy as np
import pandas as pd
import sys

In [2]:
# Import custom code
sys.path.insert(0, '..')
from scripts.custom_functions import fetch_weather, LOCAL_TIMEZONE

In [3]:
# Import data
trips_df = pd.read_csv('../data/api/fetched_stm_trip_updates.csv', low_memory=False)
schedules_df = pd.read_csv('../data/download/stop_times_2025-04-30.txt')
stops_df = pd.read_csv('../data/download/stops_2025-04-30.txt')
positions_df =  pd.read_csv('../data/api/fetched_stm_vehicle_positions.csv', low_memory=False)
alerts_df = pd.read_csv('../data/api/fetched_stm_service_alerts.csv')
weather_df = pd.read_csv('../data/api/fetched_historical_weather.csv')
traffic_df = pd.read_csv('../data/api/fetched_traffic.csv')

## Merge Data

### Schedules and stops

In [4]:
# Sort values by stop sequence
schedules_df = schedules_df.sort_values(by=['trip_id', 'stop_sequence'])

In [5]:
# Reset stop sequences (some stops might be missing)
schedules_df['stop_sequence'] = schedules_df.groupby('trip_id').cumcount() + 1

In [6]:
# Add trip progress (vehicles further along the trip are more likely to be delayed)
total_stops = schedules_df.groupby('trip_id')['stop_id'].transform('count')
schedules_df['trip_progress'] = schedules_df['stop_sequence'] / total_stops

In [7]:
# Get distribution of trip progress
schedules_df['trip_progress'].describe()

count    6.629625e+06
mean     5.139220e-01
std      2.887289e-01
min      8.547009e-03
25%      2.631579e-01
50%      5.142857e-01
75%      7.647059e-01
max      1.000000e+00
Name: trip_progress, dtype: float64

In [8]:
# Merge schedules and stops
schedules_stops_df = pd.merge(left=schedules_df, right=stops_df, how='inner', left_on='stop_id', right_on='stop_code')

In [9]:
schedules_stops_df.columns

Index(['trip_id', 'arrival_time', 'departure_time', 'stop_id_x',
       'stop_sequence', 'trip_progress', 'stop_id_y', 'stop_code', 'stop_name',
       'stop_lat', 'stop_lon', 'stop_url', 'location_type', 'parent_station',
       'wheelchair_boarding'],
      dtype='object')

In [10]:
# Rename stop id and drop other stop id columns
schedules_stops_df = schedules_stops_df.rename(columns={'stop_id_x': 'stop_id'})
schedules_stops_df = schedules_stops_df.drop(['stop_id_y', 'stop_code'], axis=1)

In [11]:
# Get coordinates of previous stop
schedules_stops_df = schedules_stops_df.sort_values(by=['trip_id', 'stop_sequence'])
schedules_stops_df['prev_lat'] = schedules_stops_df.groupby('trip_id')['stop_lat'].shift(1)
schedules_stops_df['prev_lon'] = schedules_stops_df.groupby('trip_id')['stop_lon'].shift(1)

In [12]:
# Make sure the null values are from first stops
prev_null_mask = (schedules_stops_df['prev_lat'].isna()) | (schedules_stops_df['prev_lon'].isna())
first_stop_mask = schedules_stops_df['stop_sequence'] == 1
assert prev_null_mask.sum() == first_stop_mask.sum()

In [13]:
# Get distance from previous stop
schedules_stops_df['stop_distance'] = schedules_stops_df.apply(
  lambda row: haversine((row['prev_lat'], row['prev_lon']), (row['stop_lat'], row['stop_lon']), unit=Unit.METERS),
  axis=1
)

In [14]:
# Drop previous coordinates
schedules_stops_df = schedules_stops_df.drop(['prev_lat', 'prev_lon'], axis=1)

In [15]:
# Replace null distances by zero (first stop of the trip)
schedules_stops_df['stop_distance'] = schedules_stops_df['stop_distance'].fillna(0)

In [16]:
schedules_stops_df['stop_distance'].describe()

count    6.388741e+06
mean     2.737270e+02
std      4.968967e+02
min      0.000000e+00
25%      1.699571e+02
50%      2.269746e+02
75%      2.988377e+02
max      1.376798e+04
Name: stop_distance, dtype: float64

In [71]:
# Get stop with largest distance
schedules_stops_df.iloc[schedules_stops_df['stop_distance'].idxmax()]

trip_id                                                  281377832
arrival_time                                              05:33:00
departure_time                                            05:33:00
stop_id                                                      61261
stop_sequence                                                   13
trip_progress                                                  1.0
stop_name                            YUL Aéroport Montréal-Trudeau
stop_lat                                                 45.456622
stop_lon                                                -73.751615
stop_url               https://www.stm.info/fr/recherche#stq=61261
location_type                                                    0
parent_station                                                 NaN
wheelchair_boarding                                              1
stop_distance                                          13767.98028
Name: 124692, dtype: object

After checking in the STM website, the large distance make sense because the bus stops before or after the airport are far away.

### Realtime and Scheduled Trips

In [17]:
# Convert route_id to integer
trips_df['route_id'] = trips_df['route_id'].str.extract(r'(\d+)')
trips_df['route_id'] = trips_df['route_id'].astype('int64')

In [18]:
# Sort trips
trips_df = trips_df.sort_values(by=['current_time', 'trip_id', 'route_id', 'arrival_time'])

In [19]:
# Get proportion of duplicates
subset = ['start_date', 'trip_id', 'route_id', 'stop_id']
duplicate_mask = trips_df.duplicated(subset=subset)
print(f'{duplicate_mask.mean():.2%}')

26.00%


In [20]:
# Remove duplicates
trips_df = trips_df.drop_duplicates(subset=subset, keep='last') # keep latest update

In [21]:
# Rename arrival and departure time
trips_df = trips_df.rename(columns={'arrival_time': 'rt_arrival_time','departure_time': 'rt_departure_time'})

In [22]:
# Merge trip updates with schedule
stm_trips_df = pd.merge(left=trips_df, right=schedules_stops_df, how='inner', on=['trip_id', 'stop_id'])

In [23]:
stm_trips_df.columns

Index(['current_time', 'trip_id', 'route_id', 'start_date', 'stop_id',
       'rt_arrival_time', 'rt_departure_time', 'schedule_relationship',
       'arrival_time', 'departure_time', 'stop_sequence', 'trip_progress',
       'stop_name', 'stop_lat', 'stop_lon', 'stop_url', 'location_type',
       'parent_station', 'wheelchair_boarding', 'stop_distance'],
      dtype='object')

In [ ]:
# Convert start_date to datetime
stm_trips_df['start_date'] = pd.to_datetime(stm_trips_df['start_date'], format='%Y%m%d')

In [25]:
def parse_gtfs_time(start_date:pd.Timestamp, stop_time:str) -> pd.Timestamp:
	'''
	Converts GTFS time string (e.g., '25:30:00') to datetime
	based on the arrival time.
	'''
	hours, minutes, seconds = map(int, stop_time.split(':'))
	total_seconds = hours * 3600 + minutes * 60 + seconds

	parsed_time = start_date + timedelta(seconds=total_seconds)
	return parsed_time

In [ ]:
# Parse GTFS scheduled arrival and departure time
arrival_parsed_time = stm_trips_df.apply(lambda row: parse_gtfs_time(row['start_date'], row['arrival_time']), axis=1)
departure_parsed_time = stm_trips_df.apply(lambda row: parse_gtfs_time(row['start_date'], row['departure_time']), axis=1)

In [27]:
# Convert scheduled arrival and departure time to UTC datetime
sch_arrival_time_local = arrival_parsed_time.dt.tz_localize(LOCAL_TIMEZONE)
sch_departure_time_local = departure_parsed_time.dt.tz_localize(LOCAL_TIMEZONE)

stm_trips_df['sch_arrival_time'] = sch_arrival_time_local.dt.tz_convert(timezone.utc)
stm_trips_df['sch_departure_time'] = sch_departure_time_local.dt.tz_convert(timezone.utc)

In [28]:
# Convert realtime arrival and departure time to UTC datetime
stm_trips_df['rt_arrival_time'] = pd.to_datetime(stm_trips_df['rt_arrival_time'] * 1000, origin='unix', unit='ms', utc=True)
stm_trips_df['rt_departure_time'] = pd.to_datetime(stm_trips_df['rt_departure_time'] * 1000, origin='unix', unit='ms', utc=True)

In [29]:
# Get distribution of realtime timestamps
stm_trips_df[['rt_arrival_time', 'rt_departure_time']].describe()

,rt_arrival_time,rt_departure_time
count,3284232,3284232
mean,2022-03-08 04:01:17.207861504+00:00,2022-01-26 23:57:51.159273216+00:00
min,1970-01-01 00:00:00+00:00,1970-01-01 00:00:00+00:00
25%,2025-04-29 03:10:48+00:00,2025-04-29 02:51:21+00:00
50%,2025-04-30 15:45:49.500000+00:00,2025-04-30 15:37:00+00:00
75%,2025-05-02 01:55:00+00:00,2025-05-02 01:49:00+00:00
max,2025-05-03 19:04:00+00:00,2025-05-03 19:00:00+00:00


In [30]:
# The 0 timestamps have been replaced with 1970-01-01 
zero_arrival_time = stm_trips_df['rt_arrival_time'].dt.year == 1970
zero_departure_time = stm_trips_df['rt_departure_time'].dt.year == 1970
first_stop = stm_trips_df['stop_sequence'] == 1
last_stop = stm_trips_df['trip_progress'] == 1

In [31]:
# For first stops, replace 0 departure time with scheduled time
stm_trips_df.loc[(first_stop & zero_departure_time), 'rt_departure_time'] = stm_trips_df.loc[(first_stop & zero_departure_time), 'sch_departure_time']

In [32]:
# For last stops, replace 0 arrival time with scheduled time
stm_trips_df.loc[(last_stop & zero_arrival_time), 'rt_arrival_time'] = stm_trips_df.loc[(last_stop & zero_arrival_time), 'sch_arrival_time']

In [33]:
# For other stops, replace 0 arrival time with scheduled time
stm_trips_df.loc[(~first_stop & ~last_stop & zero_arrival_time), 'rt_arrival_time'] = \
	stm_trips_df.loc[(~first_stop & ~last_stop & zero_arrival_time), 'sch_arrival_time']

In [34]:
# Calculate delay (realtime - scheduled)
# For first stops, calculate with departure time
# For the rest, calculate with arrival time
stm_trips_df['delay'] = np.where(
  	stm_trips_df['stop_sequence'] == 1,
	(stm_trips_df['rt_departure_time'] - stm_trips_df['sch_departure_time']) / pd.Timedelta(seconds=1),
  	(stm_trips_df['rt_arrival_time'] - stm_trips_df['sch_arrival_time']) / pd.Timedelta(seconds=1)
)

In [35]:
# Get distribution
stm_trips_df['delay'].describe()

count    3.284232e+06
mean     7.361182e+01
std      4.869304e+02
min     -1.359200e+04
25%      0.000000e+00
50%      0.000000e+00
75%      2.800000e+01
max      5.458500e+04
Name: delay, dtype: float64

In [36]:
stm_trips_df.columns

Index(['current_time', 'trip_id', 'route_id', 'start_date', 'stop_id',
       'rt_arrival_time', 'rt_departure_time', 'schedule_relationship',
       'arrival_time', 'departure_time', 'stop_sequence', 'trip_progress',
       'stop_name', 'stop_lat', 'stop_lon', 'stop_url', 'location_type',
       'parent_station', 'wheelchair_boarding', 'stop_distance',
       'start_date_dt', 'sch_arrival_time', 'sch_departure_time', 'delay'],
      dtype='object')

In [37]:
# Remove extra datetime columns
stm_trips_df = stm_trips_df.drop(['arrival_time', 'departure_time'], axis=1)

### Vehicle Positions

In [38]:
# Rename latitude and longitude
positions_df = positions_df.rename(columns={
  'latitude': 'vehicle_lat',
  'longitude': 'vehicle_lon',
  'status': 'vehicle_status',
  'bearing': 'vehicle_bearing',
  'speed': 'vehicle_speed',
  'timestamp': 'vehicle_dt'
})

In [39]:
# Convert vehicle timestamp to datetime
positions_df['vehicle_dt'] = pd.to_datetime(positions_df['vehicle_dt'] * 1000, origin='unix', unit='ms', utc=True)

In [ ]:
# Sort values
subset = ['trip_id', 'route_id', 'start_date', 'stop_sequence']
positions_df = positions_df.sort_values(by=subset)

In [41]:
# Get proportion of duplicates
print(f'{duplicate_mask.mean():.2%}')

26.00%


In [42]:
# Remove duplicates
positions_df = positions_df.drop_duplicates(subset=subset) # keep latest updates

In [43]:
# Merge positions
stm_trips_positions_df = pd.merge(left=stm_trips_df, right=positions_df, how='inner', on=['trip_id', 'route_id', 'start_date', 'stop_sequence'])

In [44]:
stm_trips_positions_df.columns

Index(['current_time_x', 'trip_id', 'route_id', 'start_date', 'stop_id',
       'rt_arrival_time', 'rt_departure_time', 'schedule_relationship',
       'stop_sequence', 'trip_progress', 'stop_name', 'stop_lat', 'stop_lon',
       'stop_url', 'location_type', 'parent_station', 'wheelchair_boarding',
       'stop_distance', 'start_date_dt', 'sch_arrival_time',
       'sch_departure_time', 'delay', 'current_time_y', 'vehicle_id',
       'start_time', 'vehicle_lat', 'vehicle_lon', 'vehicle_bearing',
       'vehicle_speed', 'vehicle_status', 'vehicle_dt', 'occupancy_status'],
      dtype='object')

In [ ]:
# Drop current timestamps and start_date
stm_trips_positions_df = stm_trips_positions_df.drop(['current_time_x', 'current_time_y', 'start_date'], axis=1)

In [46]:
# Calculate distance between the vehicle and the current stop
stm_trips_positions_df['vehicle_distance'] = stm_trips_positions_df.apply(
  	lambda row: haversine((row['vehicle_lat'], row['vehicle_lon']), (row['stop_lat'], row['stop_lon']), unit=Unit.METERS),
  	axis=1)

In [67]:
stm_trips_positions_df['vehicle_distance'].describe()

count    245388.000000
mean        334.559013
std        1272.973515
min           0.021027
25%           7.547791
50%         105.898779
75%         235.719620
max       19654.634122
Name: vehicle_distance, dtype: float64

In [69]:
# Get rows with largest distance
stm_trips_positions_df.nlargest(n=50, columns='vehicle_distance')[['vehicle_distance', 'delay']].describe()

,vehicle_distance,delay
count,50.000000,50.000000
mean,14557.664384,31.060000
std,1391.631155,112.981741
min,13913.765172,-175.000000
25%,13966.810660,0.000000
50%,14001.017505,0.000000
75%,14042.498183,0.000000
max,19654.634122,528.000000


The large vehicle distances don't make sense with the delays. It will confuse the model so the coordinates and the distance won't be used.

In [76]:
# Replace the vehicle speed to 0 if the vehicle is stopped
stm_trips_positions_df.loc[stm_trips_positions_df['vehicle_status'] == 1, 'vehicle_speed'] = 0

In [ ]:
# Drop unneeded columns
stm_trips_positions_df = stm_trips_positions_df.drop(['vehicle_lat', 'vehicle_lon', 'vehicle_dt', 'vehicle_distance'], axis=1)

### Service Alerts

In [77]:
# Get proportion of duplicates
duplicate_mask = alerts_df.duplicated()
print(f'{duplicate_mask.mean():.2%}')

62.65%


In [78]:
# Remove duplicates
alerts_df = alerts_df.drop_duplicates()

In [79]:
# Convert timestamps to datetime
alerts_df['start_time'] = pd.to_datetime(alerts_df['start_time'] * 1000, origin='unix', unit='ms', utc=True)
alerts_df['end_time'] = pd.to_datetime(alerts_df['end_time'] * 1000, origin='unix', unit='ms', utc=True)

In [80]:
# Fill null end time with current date (assuming the alert is still active)
alerts_df['end_time'] = alerts_df['end_time'].fillna(datetime.now(timezone.utc).replace(microsecond=0))

In [81]:
# Sort values by date
alerts_df = alerts_df.sort_values('start_time').reset_index()

In [82]:
stm_df = pd.merge(left=stm_trips_positions_df, right=alerts_df, how='left', on=['route_id', 'stop_id'])

In [83]:
stm_df.columns

Index(['trip_id', 'route_id', 'start_date', 'stop_id', 'rt_arrival_time',
       'rt_departure_time', 'schedule_relationship', 'stop_sequence',
       'trip_progress', 'stop_name', 'stop_lat', 'stop_lon', 'stop_url',
       'location_type', 'parent_station', 'wheelchair_boarding',
       'stop_distance', 'start_date_dt', 'sch_arrival_time',
       'sch_departure_time', 'delay', 'vehicle_id', 'start_time_x',
       'vehicle_bearing', 'vehicle_speed', 'vehicle_status', 'vehicle_dt',
       'occupancy_status', 'vehicle_rel_distance', 'index', 'start_time_y',
       'end_time'],
      dtype='object')

In [ ]:
# Add column has_alert
has_alert_mask = (stm_df['start_time_y'].notna()) & \
	(stm_df['rt_arrival_time'] >= stm_df['start_time_y']) & \
	(stm_df['rt_arrival_time'] <= stm_df['end_time'])
stm_df['stop_has_alert'] = has_alert_mask.astype('int64')

In [ ]:
stm_df['stop_has_alert'].value_counts()

has_alert
0    240167
1     12211
Name: count, dtype: int64

In [ ]:
# Drop unneeded datetime columns and index
stm_df = stm_df.drop(['start_time_x', 'start_time_y', 'end_time', 'index'], axis=1)

### STM and Weather

In [87]:
# Convert time string to datetime
time_dt = pd.to_datetime(weather_df['time'], utc=True)

In [88]:
# Calculate dates for weather forecast
last_day_weather = time_dt.max()
start_date = last_day_weather + timedelta(days=1)
end_date = stm_df['rt_arrival_time'].max()

In [89]:
# Fetch forecast weather
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

forecast_list = fetch_weather(start_date=start_date_str, end_date=end_date_str, forecast=True)
forecast_df = pd.DataFrame(forecast_list)

In [90]:
# Merge archive and forecast weather
weather_df = pd.concat([weather_df, forecast_df], ignore_index=True)

In [91]:
# Round arrival time to the nearest hour
rounded_arrival_dt = stm_df['rt_arrival_time'].dt.round('h')

In [92]:
# Format time to match weather data
stm_df['time'] = rounded_arrival_dt.dt.strftime('%Y-%m-%dT%H:%M')

In [93]:
# Merge STM with weather
stm_weather_df = pd.merge(left=stm_df, right=weather_df, how='inner', on='time').drop('time', axis=1)

### Traffic Data

In [94]:
# Get proportion of duplicates
duplicate_mask = traffic_df.duplicated()
print(f'{duplicate_mask.mean():.2%}')

35.91%


In [95]:
# Remove duplicates
traffic_df = traffic_df.drop_duplicates(keep='last').reset_index()

In [96]:
# Convert traffic start_time and end_time to datetime
traffic_df['start_time'] = pd.to_datetime(traffic_df['start_time'], utc=True)
traffic_df['end_time'] = pd.to_datetime(traffic_df['end_time'], utc=True)

In [97]:
# Sort by date
traffic_df = traffic_df.sort_values(by='start_time').reset_index()

In [98]:
# Fill null end times with current time (assuming the incident is still ongoing)
traffic_df['end_time'] = traffic_df['end_time'].fillna(datetime.now(timezone.utc).replace(microsecond=0))
assert traffic_df['end_time'].isna().sum() == 0

In [99]:
# Build traffic cache (for every 15 min interval)
def build_traffic_cache(traffic_df:pd.DataFrame) -> dict:
	traffic_cache = {}
	traffic_df['quarter_hour'] = traffic_df['start_time'].dt.floor('15min')

	for (hour, group) in traffic_df.groupby('quarter_hour'):
		traffic_cache[hour] = group.copy()

	return traffic_cache

Since there are many trip updates on the same day (even the same hour), there's a risk of repeating the filtering of active traffic incidents for each trip individually, which takes a lot of time for a large dataset. Traffic incidents are stable over minutes or hours. This is why the incidents will be cached by 15 minute intervals.

In [105]:
def calculate_nearby_incidents(trip_update:pd.Series, traffic_cache:dict, max_distance:int=500) -> pd.Series:
	trip_datetime = trip_update['rt_arrival_time']
	stop_coords = (trip_update['stop_lat'], trip_update['stop_lon'])

	trip_quarter_hour = trip_datetime.floor('15min')

	# Get cached incidents
	quarter_hour_incidents = traffic_cache.get(trip_quarter_hour)

	# Stop if there are no incidents for that hour
	if quarter_hour_incidents is None or quarter_hour_incidents.empty:
		return pd.Series({
			'incident_nearby': 0,
			'nearest_incident_distance': None,
			'incident_category': None,
			'incident_delay': None,
			'incident_delay_magnitude': None
		})

	# Filter for active incidents at that trip hour
	active_incidents = quarter_hour_incidents[
		(quarter_hour_incidents['start_time'] <= trip_datetime) &
		(quarter_hour_incidents['end_time'] >= trip_datetime)
	].copy()

	if active_incidents.empty:
		return pd.Series({
			'incident_nearby': 0,
			'nearest_incident_distance': None,
			'incident_category': None,
			'incident_delay': None,
			'incident_delay_magnitude': None
		})

	# Calculate distance     
	active_incidents['distance'] = active_incidents.apply(
		lambda row: haversine(stop_coords, (row['latitude'], row['longitude']), unit=Unit.METERS),
		axis=1
	)

	# Filter nearby
	nearby_incidents = active_incidents[active_incidents['distance'] <= max_distance]

	if nearby_incidents.empty:
		return pd.Series({
			'incident_nearby': 0,
			'nearest_incident_distance': None,
			'incident_category': None,
			'incident_delay': None,
			'incident_delay_magnitude': None
		})
	else:
		nearest = nearby_incidents.loc[nearby_incidents['distance'].idxmin()]
		return pd.Series({
			'incident_nearby': 1,
			'nearest_incident_distance': nearest['distance'],
			'incident_category': nearest['category'],
			'incident_delay': nearest['delay'],
			'incident_delay_magnitude': nearest['magnitude_of_delay']
		})

In [106]:
# Get traffic columns (get incidents within 2 km)
traffic_cache = build_traffic_cache(traffic_df)
traffic_cols = stm_weather_df.apply(lambda row: calculate_nearby_incidents(row, traffic_cache, 2000), axis=1)

General area traffic (within 1-2 km) could affect delays, even if it's not directly at the stop. This is why incidents within 2 km are calculated.

In [107]:
# Merge the traffic
df = pd.concat([stm_weather_df, traffic_cols], axis=1)

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223531 entries, 0 to 223530
Data columns (total 38 columns):
 #   Column                     Non-Null Count   Dtype              
---  ------                     --------------   -----              
 0   trip_id                    223531 non-null  int64              
 1   route_id                   223531 non-null  int64              
 2   start_date                 223531 non-null  int64              
 3   stop_id                    223531 non-null  int64              
 4   rt_arrival_time            223531 non-null  datetime64[ns, UTC]
 5   rt_departure_time          223531 non-null  datetime64[ns, UTC]
 6   schedule_relationship      223531 non-null  int64              
 7   stop_sequence              223531 non-null  int64              
 8   trip_progress              223531 non-null  float64            
 9   stop_name                  223531 non-null  object             
 10  stop_lat                   223531 non-null  float64     

## Clean Data

### Convert columns

In [109]:
# Get columns with two values
two_values = df.loc[:, df.nunique() == 2]
two_values.columns

Index(['wheelchair_boarding', 'vehicle_status', 'has_alert',
       'incident_nearby'],
      dtype='object')

In [ ]:
print(df['wheelchair_boarding'].value_counts())
print(df['vehicle_status'].value_counts())
print(df['incident_nearby'].value_counts())
print(df['stop_has_alert'].value_counts())

wheelchair_boarding
1    211675
2     11856
Name: count, dtype: int64
vehicle_status
2    173900
1     49631
Name: count, dtype: int64
incident_nearby
0.0    189954
1.0     33577
Name: count, dtype: int64
has_alert
0    211320
1     12211
Name: count, dtype: int64


In [111]:
# Convert columns with 2 unique values to integer
df['wheelchair_boarding'] = (df['wheelchair_boarding'] == 1).astype('int64')
df['vehicle_in_transit'] = (df['vehicle_status'] == 2).astype('int64')
df['incident_nearby'] = df['incident_nearby'].astype('int64')

### Drop columns

In [112]:
# Remove columns with constant values or with more than 50% missing values
df = df.loc[:, (df.nunique() > 1) & (df.isna().mean() < 0.5)]
df.columns

Index(['trip_id', 'route_id', 'start_date', 'stop_id', 'rt_arrival_time',
       'rt_departure_time', 'schedule_relationship', 'stop_sequence',
       'trip_progress', 'stop_name', 'stop_lat', 'stop_lon', 'stop_url',
       'wheelchair_boarding', 'stop_distance', 'start_date_dt',
       'sch_arrival_time', 'sch_departure_time', 'delay', 'vehicle_id',
       'vehicle_bearing', 'vehicle_speed', 'vehicle_status', 'vehicle_dt',
       'occupancy_status', 'vehicle_rel_distance', 'has_alert', 'temperature',
       'precipitation', 'windspeed', 'weathercode', 'incident_nearby',
       'vehicle_in_transit'],
      dtype='object')

**Other columns to drop**

`stop_url`: An url is not useful for data analysis.<br>
`vehicle_status`: It has been converted to vehicle_in_transit.<br>

In [116]:
# Drop and reorder columns
df = df[[
	'trip_id',
  	'vehicle_id',
    'vehicle_in_transit', # converted
    'vehicle_bearing',
    'vehicle_speed',
	'occupancy_status',
  	'route_id',
  	'stop_id',
    'stop_name',
  	'stop_lat',
  	'stop_lon',
    'stop_distance', # engineered
    'stop_sequence',
  	'trip_progress', # engineered
    'stop_has_alert', # engineered
  	'wheelchair_boarding', # engineered
    'schedule_relationship',
  	'rt_arrival_time', # parsed
    'sch_arrival_time', # parsed
    'rt_departure_time', # parsed
    'sch_departure_time', # parsed
    'delay', # engineered
  	'temperature',
  	'precipitation',
  	'windspeed', 
	'weathercode',
  	'incident_nearby', # engineered
]]

### Create Delay Classification

According to [STM](https://www.stm.info/en/info/networks/bus-network-and-schedules-enlightened), a vehicle is considered on time if it arrives at the stop up to 1 minute before and 3 minutes after the planned schedule.

In [117]:
# Create ranges and labels
ranges = [-np.inf, -60, 0, 180, 420, np.inf]
labels = ['Very Early', 'Early', 'On Time', 'Late', 'Very Late']

In [118]:
# Add delay category column
delay_cat = pd.cut(df['delay'], bins=ranges, labels=labels, include_lowest=True, right=False)
delay_index = df.columns.get_loc('delay')
df.insert(delay_index + 1, 'delay_cat', delay_cat)

In [119]:
# Get delay category counts
df['delay_cat'].value_counts(normalize=True)

delay_cat
On Time       0.792024
Late          0.110217
Very Late     0.044674
Very Early    0.030908
Early         0.022176
Name: proportion, dtype: float64

## Export Data

In [120]:
# Export data to CSV
df.to_csv('../data/stm_weather_traffic_merged.csv', index=False)

## End